# Dimensionality reduction
This notebook shows how to reduce the dimensionality of a dataset using projection methods and manifold learning techniques. This is an important step  It is divided into the following sections:
1. [Projection methods](#Projection-methods)
2. [Manifold learning](#Manifold-learning)

## Curse of dimensionality
Numerous machine learning challenges encompass training instances that are characterized by thousands, if not millions, of features. The sheer volume of features not only considerably slows down the training process but also complicates the task of identifying effective solutions, this is known as the **curse of dimensionality**. "Dimensionality," mean the number of features or variables that each data point has. 

Dimensionality reduction is a technique to reduce the number of features in the dataset. 
Dimensionality reduction, while beneficial in speeding up training processes, inherently involves a compromise similar to image compression (e.g., converting an image to JPEG format), where some level of information loss occurs. This trade-off can potentially affect the performance of your model, making it slightly less accurate. Additionally, implementing dimensionality reduction adds complexity to your data processing pipelines.

It's advisable to first attempt training your model with the complete dataset. Dimensionality reduction should be considered a secondary option if the training process proves to be excessively time-consuming. In certain scenarios, reducing the dataset's dimensions may actually improve model performance by eliminating noise and irrelevant details.

### Data loading and preparation

In [ ]:
%pip install -U matplotlib numpy pandas scikit-learn

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import MDS
from sklearn.manifold import Isomap
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


# Get the current working directory
current_working_directory = os.getcwd()
# Go up one level from the current working directory
parent_directory = os.path.join(current_working_directory, '..')
# Add the parent directory to sys.path
sys.path.append(parent_directory)

from src.data.data_loader import create_data_loader
from src.config import FIGURE_PATH

In [ ]:
# Load the dataset
x, y = create_data_loader().load_raw_data()
training_data = pd.concat([x, y], axis=1)
# Display basic information
print(training_data.info())
training_data.head()

## Projection methods
Projection methods in machine learning and data analysis are techniques used to reduce the dimensionality of a dataset. High-dimensional datasets can be hard to work with due to the **curse of dimensionality**, which can make analysis computationally expensive and data visualization nearly impossible. Projection methods help by transforming the original high-dimensional data into a lower-dimensional space (i.e., reducing the number of variables) while trying to preserve as much of the significant information as possible.

See more about projection methods in the [documentation](https://scikit-learn.org/stable/modules/decomposition.html#decompositions).

### Principal Component Analysis (PCA)
Principal Component Analysis (PCA) is a statistical technique primarily used for dimensionality reduction in machine learning and data analysis. By identifying patterns in data and the direction of maximum variance, PCA transforms the original variables into a new set of uncorrelated variables called principal components, ordered by the amount of variance they capture. This process not only simplifies the dataset by reducing the number of dimensions without significant loss of information but also aids in visualizing complex data, detecting underlying structures, and improving the efficiency of other machine learning algorithms.

In [ ]:
percentage_variance = 0.95

pca = PCA()
pca.fit(x, y)
cumsum = np.cumsum(pca.explained_variance_ratio_)
dimensions = np.argmax(cumsum >= percentage_variance) + 1
print(f"Number of dimensions to keep {percentage_variance}% of the variance: {dimensions}")

Note: *If the dataset is to large to fit into memory run the IncrementalPCA algorithm. Sklearn has a built-in class for this purpose.*

In [ ]:
# Plot the explained variance as a function of the number of dimensions
total_amount_of_dimensions = x.shape[1]
plt.plot(cumsum, linewidth=3)
plt.axis([0, total_amount_of_dimensions, 0, 1])
plt.plot([dimensions, dimensions], [0, percentage_variance], "k:")
plt.plot([0, dimensions], [percentage_variance, percentage_variance], "k:")
plt.plot(dimensions, percentage_variance, "ko")
plt.title("Explained variance as a function of the number of dimensions")
plt.xlabel("Dimensions")
plt.ylabel("Explained Variance")
plt.grid(True)
plt.show()

## Manifold Learning
Manifold learning is an approach to non-linear dimensionality reduction. Algorithms for this task are based on the idea that the dimensionality of many data sets is only artificially high.
Make sure the same scale is used over all features. Because manifold learning methods are based on a nearest-neighbor search, the algorithm may perform poorly otherwise.

To learn more about manifold learning, visit the [sklearn documentation](https://scikit-learn.org/stable/modules/manifold.html).
Note: *Some of the algorithms can not process text data, so it is important to convert the text data to numerical data before applying the algorithms.*

In [ ]:
def plot_reduced_data(X_reduced: pd.DataFrame, title: str, y: pd.DataFrame=y):
    plt.figure(figsize=(11,4))
    plt.title(title, fontsize=15)
    if X_reduced.shape[1] == 1:
        plt.plot(X_reduced, np.zeros(X_reduced.shape), "b.")
    if X_reduced.shape[1] == 2:
        plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y)
    if X_reduced.shape[1] > 2:
        plt.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=y)
    plt.xlabel("$z_1$", fontsize=18)
    plt.ylabel("$z_2$", fontsize=18, rotation=0)
    plt.grid(True)
    plt.savefig(f"{FIGURE_PATH}{title}.png")
    plt.show()


### Locally Linear Embedding (LLE)
LLE is a nonlinear dimensionality reduction method that preserves local relationships within the data. It assumes each data point and its neighbors lie on or close to a locally linear patch of the manifold. LLE reconstructs each point from its neighbors in the high-dimensional space and seeks to preserve these local geometries in the lower-dimensional embedding. This approach is effective for uncovering the intrinsic geometric structure of the data, making it suitable for tasks such as speech recognition and organizing large collections of photos.

In [ ]:
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_reduced = lle.fit_transform(x)
plot_reduced_data(X_reduced, "Locally Linear Embedding (LLE)")

### Multidimensional Scaling (MDS)
MDS is a technique used to analyze similarity or dissimilarity data. It aims to place each object in a low-dimensional space such that the between-object distances reflect their similarities or dissimilarities as accurately as possible. MDS is versatile, applicable to various types of input data, including distances computed from high-dimensional spaces.

NB: This might be slow to run

In [ ]:
mds = MDS(n_components=2, random_state=42)
X_reduced_mds = mds.fit_transform(x)
plot_reduced_data(X_reduced_mds, "Multi-Dimensional Scaling (MDS)")

### IsoMap
IsoMap is a technique for dimensionality reduction that extends classical Multidimensional Scaling (MDS) by incorporating geodesic distances among points. It aims to preserve the global geometry of the data by estimating the true geodesic distances between all pairs of points in the lower-dimensional space. This makes it particularly effective for unfolding nonlinear manifolds where the Euclidean distance is not an accurate measure of similarity. IsoMap is used in tasks where the intrinsic geometric structure of the data is crucial, such as in 3D shape analysis and complex data visualization.

In [ ]:
isomap = Isomap(n_components=2)
X_reduced_isomap = isomap.fit_transform(x)
plot_reduced_data(X_reduced_isomap, "Isomap")

### T-distributed Stochastic Neighbor Embedding (t-SNE)
t-SNE is a powerful nonlinear dimensionality reduction technique well-suited for visualizing high-dimensional datasets. It works by converting similarities between data points to joint probabilities and aims to minimize the divergence between these probabilities in the high-dimensional and low-dimensional spaces. 

In [ ]:
tsne = TSNE(n_components=2, random_state=42)
X_reduced_tsne = tsne.fit_transform(x)
plot_reduced_data(X_reduced_tsne, "t-SNE")

### Linear Discriminant Analysis (LDA)
LDA is both a dimensionality reduction technique and a classifier. For dimensionality reduction, LDA focuses on finding the directions (linear discriminants) that maximize the separation between multiple classes. By projecting the data onto these directions, LDA ensures that the variance between classes is maximized while the variance within classes is minimized. This makes LDA particularly useful in preprocessing steps for classification tasks and in areas like facial recognition, where maintaining class separability is crucial.

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=2)
lda.fit(x, y)
X_reduced_lda = lda.transform(x)
plot_reduced_data(X_reduced_lda, "LDA")